In [2]:
import pandas as pd
import numpy as np
import random

## Reading/Cleaning/Preparing data

In [3]:
unclean_df = pd.read_csv('agaricus-lepiota.data')
names = pd.read_csv('agaricus-lepiota.data')
unclean_df.columns = ['edible?',
              'capshape',
              'cap-surface',
              'cap-color',
              'bruises?',
              'odor',
              'gill-attachment',
              'gill-spacing',
              'gill-size',
              'gill-color',
              'stalk-shape',
              'stalk-root',
              'stalk-surface-above-ring',
              'stalk-surface-below-ring',
              'stalk-color-above-ring',
              'stalk-color-below-ring',
              'veil-type',
              'veil-color',
              'ring-number',
              'ring-type',
              'spore-print-color',
              'population',
              'habitat'
]

In [4]:
# looking at the raw data, we can see that variable 16 is all classified the same
test = unclean_df['veil-type'].unique()
print(test)
# We can remove this column as it is not useful for prediction.
unclean_df2 = unclean_df.drop('veil-type', axis=1)

['p']


In [5]:
# remove missing data
df = unclean_df2.loc[(unclean_df2['stalk-root'] != '?')]

In [6]:
# split data into train and test set
def test_train_split(df, test_proportion):

    if isinstance(test_proportion, float):
        test_proportion = round(test_proportion * len(df))
    
    rows = df.index.tolist()
    test_rows = random.sample(population=rows, k=test_proportion)
    test_set = df.loc[test_rows]
    train_set = df.drop(test_rows)
    
    return train_set, test_set

# get train and test set
# use different proportions to test for different models
random.seed(2)
train_df, test_df = test_train_split(df, 0.33)
print("Length of train dataset: {}, length of test dataset: {}".format(len(train_df), len(test_df)))

Length of train dataset: 3781, length of test dataset: 1862


In [7]:
# turn dataframe into array for other functions to use
# numpy array more efficient than dataframe
train_data = train_df.values
test_data = test_df.values

## Functions for Training Procedure


In [8]:
# Check if at a node, all mushrooms are the same (perfectly classified)
def check_mushroom_edibility(data):
    y_vals = data[:,0]
    unique_classes = np.unique(y_vals)
    if len(unique_classes) != 2:
        return True
    else:
        return False

In [9]:
# find all possible ways to test the data
def get_possible_splits(data):
    splits = {}
    _, n_col = data.shape
    
    for col_index in range(n_col):
        vals = data[:, col_index]
        unique_val = np.unique(vals)
        splits[col_index] = unique_val

    return splits

In [10]:
# calculate expected entropy of a variable
def calculate_entropy(data, split_col_index, split_vals_list):
    entropy_list = []
    count_list = []
    for value in split_vals_list:
        attribute_data = data[(data[:,split_col_index]) == value]
        _, counts = np.unique(attribute_data[:,0], return_counts=True)
        probs = counts / counts.sum()
        entropy = sum(-probs * np.log2(probs))
        entropy_list.append(entropy)
        count_list.append(len(attribute_data))
    expected_entropy = 0
    for i in range(len(entropy_list)):
        expected_entropy += entropy_list[i] * count_list[i] / len(data)
    return expected_entropy

In [11]:
# find the best possible split
def find_best_split(data, possible_splits, headers):
    # loop over all variables
    entropy_list = []
    for i in range(1,len(possible_splits)):
        entropy = calculate_entropy(data, i, possible_splits[i])
        entropy_list.append(entropy)
    index = np.array(entropy_list).argmin()
    best_split = headers[index+1]
    return best_split, index+1

In [12]:
# determine if the leaf node is predominantly edible or poisonous.
def classifier(data):
    labels = data[:,0]
    edibility, indices = np.unique(labels, return_counts=True)
    index = indices.argmax()
    classification = edibility[index]
    return classification

## Main Algorithm

In [13]:
def decisiontreealgorithm(data, stopping_depth, counter=0, headers=0):
    # keep track of headers for entropy calculation
    if counter == 0:
        headers = train_df.columns

    # base case
    if counter == stopping_depth or check_mushroom_edibility(data):
        label = classifier(data)
        return label
    
    # inductive step
    else:
        counter += 1
        # find highest information gain variable to split on
        possible_splits = get_possible_splits(data)
        best_split, index = find_best_split(data, possible_splits, headers)
        headers = np.delete(headers, index)

        # build tree
        tree = {}
        attribute_list = possible_splits[index]

        # interate through all attributes, appending new subtree each time
        for attribute in attribute_list:
            details = "Split feature {}, on class: {}".format(best_split, attribute)
            old_attribute_data = data[data[:,index] == attribute]
            # remove variable which is now useless for prediction
            attribute_data = np.delete(old_attribute_data, index, axis=1)
            subtree = decisiontreealgorithm(attribute_data, stopping_depth, counter, headers)
            tree[details] = [subtree]

        return tree

In [14]:
decisiontreealgorithm(train_data, 2)

{'Split feature odor, on attribute: a': ['e'],
 'Split feature odor, on attribute: c': ['p'],
 'Split feature odor, on attribute: f': ['p'],
 'Split feature odor, on attribute: l': ['e'],
 'Split feature odor, on attribute: m': ['p'],
 'Split feature odor, on attribute: n': [{'Split feature spore-print-color, on attribute: k': ['e'],
   'Split feature spore-print-color, on attribute: n': ['e'],
   'Split feature spore-print-color, on attribute: r': ['p'],
   'Split feature spore-print-color, on attribute: w': ['e']}],
 'Split feature odor, on attribute: p': ['p']}

In [15]:
decisiontreealgorithm(train_data, 3)

{'Split feature odor, on attribute: a': ['e'],
 'Split feature odor, on attribute: c': ['p'],
 'Split feature odor, on attribute: f': ['p'],
 'Split feature odor, on attribute: l': ['e'],
 'Split feature odor, on attribute: m': ['p'],
 'Split feature odor, on attribute: n': [{'Split feature spore-print-color, on attribute: k': ['e'],
   'Split feature spore-print-color, on attribute: n': ['e'],
   'Split feature spore-print-color, on attribute: r': ['p'],
   'Split feature spore-print-color, on attribute: w': [{'Split feature cap-color, on attribute: c': ['e'],
     'Split feature cap-color, on attribute: g': ['e'],
     'Split feature cap-color, on attribute: n': ['e'],
     'Split feature cap-color, on attribute: p': ['e'],
     'Split feature cap-color, on attribute: w': ['p'],
     'Split feature cap-color, on attribute: y': ['p']}]}],
 'Split feature odor, on attribute: p': ['p']}

In [16]:
decisiontreealgorithm(train_data, 4)

{'Split feature odor, on attribute: a': ['e'],
 'Split feature odor, on attribute: c': ['p'],
 'Split feature odor, on attribute: f': ['p'],
 'Split feature odor, on attribute: l': ['e'],
 'Split feature odor, on attribute: m': ['p'],
 'Split feature odor, on attribute: n': [{'Split feature spore-print-color, on attribute: k': ['e'],
   'Split feature spore-print-color, on attribute: n': ['e'],
   'Split feature spore-print-color, on attribute: r': ['p'],
   'Split feature spore-print-color, on attribute: w': [{'Split feature cap-color, on attribute: c': ['e'],
     'Split feature cap-color, on attribute: g': ['e'],
     'Split feature cap-color, on attribute: n': ['e'],
     'Split feature cap-color, on attribute: p': ['e'],
     'Split feature cap-color, on attribute: w': ['p'],
     'Split feature cap-color, on attribute: y': ['p']}]}],
 'Split feature odor, on attribute: p': ['p']}

## Classification Step

In [17]:
# Classification step
# Pass in our instance and our trained model, output the label (edible = e, poisonous = p)
def classify(instance, model):
    already_classified = [["e"], ["p"]]
    # remove outer list when variable splits
    if isinstance(model, dict):
        split_det = list(model.keys())[0]
        feature_index = split_det.find(",")
    else:
        [unpack_model] = model
        split_det = list(unpack_model.keys())[0]
        feature_index = split_det.find(",")
        model = unpack_model
        
    # extract root feature and split value
    model_feature = str(split_det)[14:feature_index]
    model_answer = str(split_det)[-1:]

    # base case
    if str(instance[model_feature]) == model_answer:
        if model[split_det] in already_classified:
            return model[split_det]
        return classify(instance, model[split_det])

    # inductive step
    # if instance feature does not match with model, delete model question and iterate through again
    else:
        dummy_model = model.copy()
        dummy_model.pop(split_det)
        return classify(instance, dummy_model)

In [24]:
# Classify test set
def accuracy_testing(df, model):
    incorrect = 0
    incorrect_indices = []
    for index, inst in df.iterrows():
        label = classify(inst, model)
        if label[0] != inst[0]:
            incorrect += 1
            incorrect_indices.append(index)
    incorrect_df = df[df.index.isin(incorrect_indices)]
    return incorrect, incorrect_df

In [25]:
# testing the model
model = decisiontreealgorithm(test_data, 2)
incorrect_no, incorrect_df = accuracy_testing(test_df, model)
incorrect_df[["edible?","odor","spore-print-color","cap-color"]]

,edible?,odor,spore-print-color,cap-color
7599,p,n,w,y
7738,p,n,w,y
5127,p,n,w,w
5280,p,n,w,w
7482,p,n,w,y


In [23]:
# train_error - test_error = approximate_error
model = decisiontreealgorithm(test_data, 2)
test_no, _ = accuracy_testing(test_df, model)
train_no, _ = accuracy_testing(train_df, model)
train_no / len(train_df) - test_no / len(test_df)

0.000223998618225391